In [ ]:
# Lectura del archivo .csv de training
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')
plaintext_rdd = sc.textFile('muestra100.csv')
dataframe = pycsv.csvToDataFrame(sqlCtx, plaintext_rdd, parseDate=False)


data = dataframe.rdd
data


In [ ]:
from compresores import *

def parsearReview(x):
    parser = SymbolParser('letra')
    texto = x[0]
    tabla = x[1]
    parser.parsearTexto(texto, tabla)
    return tabla

def mergeSymbolTables(tablaA, tablaB):
    # Copio A en C
    tablaC = SymbolTable()
    for simb in tablaA.verSimbolos():
        freq = tablaA.verFrecuencia(simb)
        while (freq > 1):
            tablaC.aumentarFrecuencia(simb)
            freq -= 1  
    # Veo ahora los simbolos en B
    for simb in tablaB.verSimbolos():
        freq = tablaB.verFrecuencia(simb)
        while (freq > 1):
            tablaC.aumentarFrecuencia(simb)
            freq -= 1    
    return tablaC

# cmpr es una lista de compresores estáticos, donde el compresor i
# es encargado de comprimir reviews de puntaje i+1
cmpr = []
for i in range(0,5):
    cmpr.append(CompresorAritmetico('letra'))  
        
# Del set de datos, obtengo para compresor i las reviews de
# puntaje i+1 y las parseo para actualizar las tablas de símbolos
for i in range(0,5):
    reviews = data.filter(lambda x: x.Prediction == (i+1)).map(lambda x: x.Text)
    reviews = reviews.map(lambda x: (x, SymbolTable())).map(parsearReview)
    # Ahora en reviews hay tablas de símbolos que representan
    # las reviews originales. ¡Las mergeamos!
    tablaFinal = reviews.reduce(mergeSymbolTables)
    cmpr[i].asignarSymbolTable(tablaFinal)

# Pruebo mostrando los simbolos de cada compresor
for i in range(0,5):
    print 'Símbolos del conversor', i, ':', cmpr[i].getSymbolTable().verSimbolos()
    print 'Frecuencia de la a:', cmpr[i].getSymbolTable().verFrecuencia('a')


In [ ]:
# DE ACA PARA ABAJO ZONA DE TESTS
# NO DAR MUCHA BOLA



print lasReviews.takeOrdered(1)[0][1]

In [ ]:
q = lasReviews.map(lambda x: (x[0],x[1].count("!!! "))).reduceByKey(lambda x, y: x+y)
print q.collect()

In [ ]:
textos = lasReviews.map(lambda x: (x[1], -1)).reduceByKey(lambda x, y: x+y).map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x+y)
print textos.takeOrdered(25)

In [ ]:
from enchant.checker import SpellChecker
chkr = SpellChecker("en_US")

reviewsAlAzar = textos.takeSample(False, 5)


for r in range(0,5):
    print reviewsAlAzar[r]
    chkr.set_text(reviewsAlAzar[r])
    for err in chkr:
        print "ERROR! Wrong word:", err.word